# Training Workbook for Supercomputer Environment

## Setup Global Variables and Harm2d Functions

In [6]:
## Setup and configs
# imports
import os
import time
import matplotlib
import matplotlib.pyplot as plt
# global variables
global notebook
global axisym,set_cart,axisym,REF_1,REF_2,REF_3,set_cart,D,print_fieldlines
global lowres1,lowres2,lowres3, RAD_M1, RESISTIVE, export_raytracing_GRTRANS, export_raytracing_RAZIEH,r1,r2,r3
global r_min, r_max, theta_min, theta_max, phi_min,phi_max, do_griddata, do_box, check_files, kerr_schild

global do_train
## NOTE toggle do_train to run training on setup
do_train = False

notebook = 1

# total data is shape (10000, 224, 48, 96)
harm_directory = os.environ['HOME']+f'/bh/harm2d'
os.chdir(harm_directory)

print(f'Running setup scripts...')
start_time = time.time()
%run -i setup.py build_ext --inplace
%run -i pp.py build_ext --inplace
print(f"Execution time: {time.time() - start_time}")

# set params
lowres1 = 1 # 
lowres2 = 1 # 
lowres3 = 1 # 
r_min, r_max = 1.0, 100.0
theta_min, theta_max = 0.0, 9
phi_min, phi_max = -1, 9
do_box=0
set_cart=0
set_mpi(0)
axisym=1
print_fieldlines=0
export_raytracing_GRTRANS=0
export_raytracing_RAZIEH=0
kerr_schild=0
DISK_THICKNESS=0.03
check_files=1
notebook=1
interpolate_var=0
AMR = 0 # get all data in grid

print('Imports and setup done.')
%matplotlib inline

Running setup scripts...
Execution time: 5.834523439407349
Imports and setup done.


In [ ]:
import os
import time
harm_directory = os.environ['HOME']+f'/bh/harm2d'
os.chdir(harm_directory)

print(f'Running async setup scripts...')
start_time = time.time()
%run -i async_read.py build_ext --inplace
print(f"Execution time: {time.time() - start_time}")

## Visualization

In [ ]:
print(f'Running setup scripts...')
start_time = time.time()
%run -i async_read.py build_ext --inplace
print(f"Execution time: {time.time() - start_time}")
%matplotlib inline

In [7]:
import os
import yaml
from tqdm import tqdm
import time

os.chdir(os.environ['HOME'] + '/bh/harm2d')

from models.cnn.threed_cnn import *

# access device, cuda device if accessible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# load in model
model = B3_CNN().to(device)
model_path = os.environ['HOME'] + "/bh/harm2d/models/cnn/saves/b3_v0.1.0.pth"
print(f'Rendering using model at: {model_path} \n')

# load in trained weights
loaded_temp = torch.load(f=model_path)
model.load_state_dict(loaded_temp)
model.eval()

# load in configs
with open('train_config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# change to location of dumps
dumps_path = '/pscratch/sd/l/lalakos/ml_data_rc300/reduced'
os.chdir(dumps_path)

# get first frame to emulate off of
first_frame_index = config['start_dump']

# process first frame into tensor
# rblock_new_ml()
# rpar_new(first_frame_index)
# # get grid data
# rgdump_griddata(dumps_path)
# rdump_griddata(dumps_path, first_frame_index)
# data_tensor = tensorize_globals(rho=np.log10(rho), ug=np.log10(ug), uu=uu, B=B)

# get saved first frame
save_path = os.environ['HOME']+'/bh/data.pkl'
data_tensor = torch.load(f=save_path)

# frame creation configs configs
num_frames = 50
save_path = os.environ['HOME']+f'/bh/movies/sc_pred_frames/'
print(f'Saving {num_frames} at: {save_path}.')

# make frames
prog_bar = tqdm(range(first_frame_index, first_frame_index+num_frames), total=num_frames)
for index in prog_bar:
    # plot and save
    frame_time_start = time.time()
    
    plc_cart_ml(
        var=(data_tensor[0][0].cpu().unsqueeze(0).detach().numpy()), 
        min=-2,
        max=2, 
        rmax=100, 
        offset=0, 
        name=save_path+f'pred_rho_{index}', 
        label=r"$\sigma r {\rm sin}\theta$ at dump %d" % index,
    )
    
    # send data to device
    data_tensor = data_tensor.to(device)
    
    # predict without tracking gradients
    data_tensor = model.inference(data_tensor)

    prog_bar.set_description(f'Frame {index} computed and saved in {time.time()-frame_time_start:.4f} s')

print(f'{num_frames} frames saved.')

Rendering using model at: /global/homes/a/arjuna/bh/harm2d/models/cnn/saves/b3_v0.1.0.pth 

Saving 50 at: /global/homes/a/arjuna/bh/movies/sc_pred_frames/.


  0%|          | 0/50 [00:00<?, ?it/s]


NameError: name 'r' is not defined

<Figure size 6400x3200 with 0 Axes>